<a href="https://colab.research.google.com/github/Alan-B525/TP_FINAL_IC2022/blob/main/RNN_TP_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redes Neuronales Recurrentes

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


# Preprocesado de datos

Funcion para preprocesamiento de los dataset

In [38]:
scaler = MinMaxScaler( feature_range=(0, 1) )

ve = 120


In [39]:

def preprocessing(data):
  data = data.drop(['Date','Adj Close'], axis=1)
  data = data.round(2)

  #transform dataframe to numpy array
  data = data.values

  #normalize data
  data = scaler.fit_transform(data)

  X_train = []
  y_train = []

  for i in range(ve, len(data)):
      X_train.append(data[i-ve:i, :])
      y_train.append(data[i, 0])
 
  X_train, y_train = np.array(X_train), np.array(y_train)
  


  return X_train, y_train

In [40]:
#importar datos
dataset_train = pd.read_csv('NASDAQ_train.csv')

#se llama la funcion de procesamiento
X_train, y_train = preprocessing(dataset_train)


# Construcción de la RNN

In [41]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

### Inicialización del la red

In [42]:
regressor = Sequential()

### Añadir las capas  LSTM y  regularización por Dropout

In [43]:
regressor.add(LSTM(units=60, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(rate=0.2))
regressor.add(LSTM(units=60, return_sequences=True))
regressor.add(Dropout(rate=0.2))
regressor.add(LSTM(units=60, return_sequences=True))
regressor.add(Dropout(rate=0.2))
regressor.add(LSTM(units=60, return_sequences=True))
regressor.add(Dropout(rate=0.2))
regressor.add(LSTM(units=60, return_sequences=True))
regressor.add(Dropout(rate=0.2))
regressor.add(LSTM(units=60, return_sequences=True))
regressor.add(Dropout(rate=0.2))
regressor.add(LSTM(units=60, return_sequences=True))
regressor.add(Dropout(rate=0.2))
regressor.add(LSTM(units=60, return_sequences=False))
regressor.add(Dropout(rate=0.2))

### Añadir la capa de salida

In [44]:
regressor.add(Dense(units=1))

### Compilar la RNN 

In [45]:
regressor.compile(optimizer='adam', loss='mean_squared_error')

### Entrenamiento de la Red Neuronal Recurrente

In [ ]:
regressor.fit(X_train, y_train, epochs=50, batch_size=32, verbose=False)

# Predecir el valor de las acciones

## Obtener el valor real de las acciones desde enero 22 a mayo 22


In [ ]:

dataset_test = pd.read_csv('NASDAQ_test.csv')

X_test, y_test = preprocessing(dataset_test)

real_stock_price = dataset_test.loc[ve:, ['Open']].values.round(2)



Realizamos la predicción.

In [ ]:
# Prediccion de los valores del mes de enero 
predicted_stock_price = regressor.predict(X_test)


#Transformación de los precios

In [ ]:
price_t = []

for i in range(len(predicted_stock_price)):
    x = (predicted_stock_price[i,0]*(scaler.data_max_[0]-scaler.data_min_[0]))+scaler.data_min_[0]
    price_t.append(x)


# Visualizar los resultados

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Real Stock Price')
plt.plot(price_t, color = 'blue', label = 'Predicted Stock Price')
plt.title('Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

Error absoluto medio de la predicción.

In [ ]:
mean_absolute_error(predicted_stock_price, real_stock_price)

Valos promedio de las acciones el mes de Enero.

In [ ]:
np.mean(real_stock_price)

El porcentaje de error sobre el valor de las acciones.

In [ ]:
mean_absolute_error(predicted_stock_price, real_stock_price) / np.mean(real_stock_price)

Calcularmos el rrror de cada dia de enero y lo graficamos.

In [ ]:
error = predicted_stock_price - real_stock_price

In [ ]:
plt.plot(error)
plt.axhline(y=0, xmin=0, xmax=20, ls='--', color='k')
plt.title('Trading Sistem')
plt.xlabel('Time')
plt.ylabel('Prediction error')